# Purpose:

# 1) download datasets

In [4]:
# Initial imports.
%run s0_fifa_players_functions.ipynb

from pathlib import Path
import os

## ---> Modify the location below to fit your dev environment

In [5]:
os.chdir('E:\\data-engineering@ColumbiaUnv\\Module-20_final_project\\resources')

!curl -L https://raw.githubusercontent.com/ponda614/Data-Science-Final-Project-/main/resources/players.csv  -o fifa_players_git.csv

os.chdir('E:\\data-engineering@ColumbiaUnv\\Module-20_final_project\\notebooks')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3879k  100 3879k    0     0  4923k      0 --:--:-- --:--:-- --:--:-- 4923k


# 2) build dataframes

In [6]:
players_dataset_loc = os.path.join('../resources', 'fifa_players_git.csv')
players_df = create_fifa_players_dataframe(players_dataset_loc)
#goal_keepers_dataset_loc= os.path.join('../resources', 'fifa_goal_keepers_git.csv')
#goal_keepers_df = create_fifa_players_dataframe(goal_keepers_dataset_loc)
# display(goal_keepers_df)

# convert rating from numerical to categorical values
players_df['Rating'] = players_df['Rating'].apply(adjust_rating)

# display(players_df)

# 3) Define the features set

In [8]:
players_stats_df=build_players_stats_dataframe(players_df)

players_stats_df = encode_field(players_stats_df, 'rating')

players_dataset_loc= os.path.join('../resources', 'fifa_players_encoded.csv')
players_stats_df.to_csv(players_dataset_loc, index=False)

# display(players_stats_df)
X = players_stats_df.copy()
X = X.drop(["rating"], axis=1)
# display(X)

Y = players_stats_df['rating'].values

# display(array)
# # display(players_stats_df)
# display(Y)
# display(X)

C:\Users\albat\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\albat\anaconda3\envs\mlenv\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


# Splitting into Train and Test sets.

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=78)
# y_train = y.train.ravel()
# display(X_train)
# display(X_test)
# display(y_train)
# display(y_test)

In [10]:
from sklearn.preprocessing import StandardScaler

# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# display(y_train)
# display(X_train_scaled)
# display(X_test_scaled)

# Fit the Random Forest Model

In [13]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators=128, random_state=78) 
# Fitting the model
random_forest_model = random_forest_model.fit(X_train_scaled, y_train.ravel())
display(random_forest_model)

RandomForestClassifier(n_estimators=128, random_state=78)

# Making predictions using the Test Data

In [18]:
predictions = random_forest_model.predict(X_test_scaled)
display(predictions)

array([2, 2, 0, ..., 2, 2, 0])

# Evaluate the Model

In [19]:
import pandas
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)
# display(cm)
# print(y_test.shape)
# print(y_test.shape)
y_test
predictions

# Create a DataFrame from the confusion matrix.
cm_df = pandas.DataFrame(cm, index=["Actual 1", "Actual 2", "Actual 3"], columns=["Predicted 1", "Predicted 2", "Predicted 3"])

display(cm_df)

,Predicted 1,Predicted 2,Predicted 3
Actual 1,2455,0,142
Actual 2,0,17,11
Actual 3,334,2,1203


# Calculating the accuracy score

In [20]:
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"\nAccuracy Score : {acc_score}")
print(f"\nClassification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 1,Predicted 2,Predicted 3
Actual 1,2455,0,142
Actual 2,0,17,11
Actual 3,334,2,1203


Accuracy Score : 0.8825648414985591
Classification Report
              precision    recall  f1-score   support

           0       0.88      0.95      0.91      2597
           1       0.89      0.61      0.72        28
           2       0.89      0.78      0.83      1539

    accuracy                           0.88      4164
   macro avg       0.89      0.78      0.82      4164
weighted avg       0.88      0.88      0.88      4164



# Rank the features

In [21]:
# Calculate feature importance in the Random Forest model.
importances = random_forest_model.feature_importances_
importances

array([0.01875293, 0.02866747, 0.02587341, 0.07050612, 0.01779059,
       0.06911345, 0.01600435, 0.01585225, 0.02949941, 0.09155757,
       0.02121931, 0.02189087, 0.0157765 , 0.13121651, 0.01402036,
       0.02141166, 0.01611114, 0.01632482, 0.01672684, 0.01788289,
       0.01595695, 0.04442765, 0.02732456, 0.03299448, 0.01484165,
       0.06298452, 0.03407117, 0.05808484, 0.03311572])

In [22]:
# We can sort the features by their importance.
sorted(zip(random_forest_model.feature_importances_, X.columns), reverse=True)

[(0.13121650811489446, ('movement_reactions',)),
 (0.09155757134515508, ('skill_ball_control',)),
 (0.07050612004137573, ('attacking_short_passing',)),
 (0.0691134476782312, ('skill_dribbling',)),
 (0.06298451765983044, ('mental_composure',)),
 (0.05808484467143444, ('defending_standing_tackle',)),
 (0.04442765116238057, ('mental_interceptions',)),
 (0.03407117425729655, ('defending_marking',)),
 (0.033115721961332126, ('defending_sliding_tackle',)),
 (0.03299447905471283, ('mental_vision',)),
 (0.029499409029997797, ('skill_long_passing',)),
 (0.02866747329547948, ('attacking_finishing',)),
 (0.027324557161065802, ('mental_positioning',)),
 (0.025873409577133837, ('attacking_heading_accuracy',)),
 (0.021890872461281744, ('movement_sprint_speed',)),
 (0.021411657888018475, ('power_shot_power',)),
 (0.021219313445546267, ('movement_acceleration',)),
 (0.018752925296811253, ('attacking_crossing',)),
 (0.01788288609744496, ('power_long_shots',)),
 (0.017790593705748092, ('attacking_volley